# Create covariate dataset

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pickle
%matplotlib inline
plt.style.use('seaborn')

In [2]:
import os
cwd = os.getcwd()

## Cleaning data

In [7]:
# covariates
df = pd.read_stata('../data/cty_full_covariates.dta')
print(df.shape) # 3138 counties

(3138, 83)


In [8]:
df.columns # select columns

Index(['cty', 'county_name', 'cty_pop2000', 'cz', 'cz_pop2000', 'statename',
       'state_id', 'stateabbrv', 'csa', 'csa_name', 'cbsa', 'cbsa_name',
       'intersects_msa', 'poor_share', 'cs_race_theil_2000', 'cs00_seg_inc',
       'cs00_seg_inc_pov25', 'cs00_seg_inc_aff75', 'frac_traveltime_lt15',
       'hhinc00', 'inc_share_1perc', 'gini99', 'frac_middleclass', 'taxrate',
       'subcty_total_taxes_pc', 'subcty_exp_pc', 'tax_st_diff_top20',
       'ccd_exp_tot', 'ccd_pup_tch_ratio', 'score_r', 'dropout_r',
       'num_inst_pc', 'tuition', 'gradrate_r', 'cs_labforce', 'cs_elf_ind_man',
       'mig_inflow', 'mig_outflow', 'cs_born_foreign', 'scap_ski90pcm',
       'rel_tot', 'crime_total', 'cs_fam_wkidsinglemom', 'median_house_value',
       'median_rent', 'unemp_rate', 'e_rank_b', 'cz_name', 'pop_d_2000_1980',
       'lf_d_2000_1980', 'cs_educ_ba', 'cs_frac_black', 'cs_frac_hisp',
       'puninsured2010', 'primcarevis_10', 'amb_disch_per1000_10',
       'diab_hemotest_10', 'diab_ey

In [9]:
# renaming variables 

names = ['cty', 'county_name', 'cty_pop2000', 'statename', 'stateabbrv', 
        'pop_density', 'gini99', 's_rank', 'e_rank_b', 'cs00_seg_inc', 
         'cs_race_theil_2000', 'hhinc00', 'poor_share', 'frac_middleclass', 
        'mig_inflow', 'mig_outflow', 'cs_born_foreign', 'rel_tot', 'crime_total',
         'puninsured2010',  'cs_labforce', 'unemp_rate', 'cs_frac_black', 'cs_frac_hisp',
         'bmi_obese', 'cur_smoke','exercise_any', 'median_house_value', 'cs_educ_ba', 
         'reimb_penroll_adj10', 'subcty_exp_pc'
        ]

In [10]:
nnames = ['county', 'county_name', 'population', 'statename', 'stateabbrv', 
        'density', 'gini', 'relative_mob', 'absolute_mob', 'segregation_income', 
          'segregation_race', 'income', 'poverty', 'middle_class', 'mig_inflow', 
          'mig_outflow', 'foreign', 'religion', 'crime_rate', 'uninsured', 
        'labor_force', 'unemployment', 'pct_black', 'pct_hispanic', 
          'obesity', 'smoking', 'exercise', 'house_value', 'college', 'medicare_expenses', 
        'local_gov_exp'
         ]


d = dict(zip(names, nnames))
df.rename(columns=d, inplace=True)
df = df.loc[:, nnames]

In [11]:
df.head()

,county,county_name,population,statename,stateabbrv,density,gini,relative_mob,absolute_mob,segregation_income,...,unemployment,pct_black,pct_hispanic,obesity,smoking,exercise,house_value,college,medicare_expenses,local_gov_exp
0,1001,Autauga,43671,Alabama,AL,73.277412,0.379976,40.602394,41.872879,0.036455,...,0.037379,17.008999,1.396808,0.276190,0.216981,0.619048,126368.4,18.0,9489.02,1059.669312
1,1003,Baldwin,140415,Alabama,AL,87.960236,0.489738,25.488312,44.492390,0.032571,...,0.039112,10.224691,1.756222,0.223032,0.215395,0.729490,163292.5,23.1,9618.34,2209.910400
2,1005,Barbour,29038,Alabama,AL,32.814877,0.490637,42.124779,37.718758,0.021680,...,0.068132,46.039669,1.646119,0.412281,0.189655,0.603448,91443.8,10.9,9761.77,1570.248291
3,1007,Bibb,20826,Alabama,AL,33.427227,0.417098,45.493622,42.245155,0.017462,...,0.061639,22.010948,1.008355,0.322581,0.218750,0.554688,99441.8,7.1,11269.81,1338.441162
4,1009,Blount,51024,Alabama,AL,79.035255,0.333042,25.050438,49.043320,0.013379,...,0.032847,1.171997,5.326905,0.354167,0.265306,0.602041,115704.4,9.6,10238.20,987.438843


In [12]:
df.duplicated(['statename','county']).value_counts()

False    3138
dtype: int64

# Missing data

In [13]:
from utils import utils as f

In [14]:
f.missing_data(df)

exercise             0.237731
obesity              0.235819
smoking              0.235182
relative_mob         0.084449
absolute_mob         0.084449
crime_rate           0.056405
gini                 0.032505
mig_inflow           0.029318
mig_outflow          0.029318
medicare_expenses    0.002549
uninsured            0.001275
unemployment         0.001275
religion             0.000637
pct_black            0.000319
middle_class         0.000319
pct_hispanic         0.000319
college              0.000319
local_gov_exp        0.000319
dtype: float64

In [16]:
# impute values
f.missing_data(f.impute_values(df, group_vars=['statename']))

exercise         0.237731
obesity          0.235819
smoking          0.235182
absolute_mob     0.084449
relative_mob     0.084449
crime_rate       0.056405
gini             0.032505
unemployment     0.001275
uninsured        0.001275
religion         0.000637
local_gov_exp    0.000319
college          0.000319
pct_hispanic     0.000319
pct_black        0.000319
middle_class     0.000319
dtype: float64

In [18]:
# plots continuous variables
variables = ['population', 'density', 'crime_rate', 'poverty', 'middle_class', 
            'mig_inflow', 'mig_outflow', 'foreign', 'pct_black', 'pct_hispanic', 
            'religion', 'uninsured', 'medicare_expenses', 'house_value', 'local_gov_exp', 
            'college', 'obesity', 'exercise', 'smoking', 'unemployment', 
            'labor_force', 'income',  'segregation_income', 'segregation_race']

len(variables)

24

In [ ]:
sns.kdeplot(df.loc[:, 'local_gov_exp']);
# sns.kdeplot(df.loc[:, variables[23]]);
# sns.kdeplot(np.log(df.loc[:, variables[23]]), label='log');

In [28]:
log_variables = ['population', 'crime_rate', 'poverty', 'mig_inflow', 
            'mig_outflow', 'foreign', 'pct_black', 'pct_hispanic', 
            'house_value', 'local_gov_exp', 'unemployment', 'income']
len(log_variables)

12

In [20]:
df_tr = f.transform_variables(df, variables=log_variables, kind='log')

In [24]:
z_variables = ['gini', 'relative_mob', 'absolute_mob', 
            'middle_class', 'segregation_income', 'religion', 
            'labor_force', 'uninsured', 'medicare_expenses', 
            'college', 'obesity', 'smoking', 'exercise' ]
df_tr = f.transform_variables(df_tr, variables=z_variables, kind='z')

In [25]:
len(log_variables)+len(z_variables)

25

In [29]:
df_tr.duplicated(['county']).sum()

0

In [39]:
# subset variables

# -Demographic: % Black, Hispanic (since Chetty uses this to race adjust)
# -Social: Crime rate, segregation
# -Economic: Gini, possibly unemployment
# -Health Access: uninsured, medicare expenses

subset_vars = ['county', 'county_name', 'stateabbrv', 
               'statename', 'z_relative_mob', 'z_gini', 'log_population', 'log_income', 'z_segregation_income', 'log_unemployment',
               'z_uninsured', 'z_medicare_expenses', 'log_crime_rate', 'log_pct_black', 
               'log_pct_hispanic', 'z_obesity', 'z_smoking', 'z_exercise'] 

In [40]:
df_sel = df_tr.loc[:, subset_vars]

In [41]:
df_sel.columns

Index(['county', 'county_name', 'stateabbrv', 'statename', 'z_relative_mob',
       'z_gini', 'log_population', 'log_income', 'z_segregation_income',
       'log_unemployment', 'z_uninsured', 'z_medicare_expenses',
       'log_crime_rate', 'log_pct_black', 'log_pct_hispanic', 'z_obesity',
       'z_smoking', 'z_exercise'],
      dtype='object')

In [42]:
df_sel.to_csv('../data/cov_sel.csv', index=False)

In [44]:
f.missing_data(df_sel)

z_exercise             0.237731
z_obesity              0.235819
z_smoking              0.235182
z_relative_mob         0.084449
z_gini                 0.032505
z_medicare_expenses    0.002549
z_uninsured            0.001275
dtype: float64